# Pandas

# Target de todas las combinaciones producto-cliente desde el periodo en que se dio la primer interaccion producto-cliente

In [1]:
import pandas as pd
import gc

# Cargar datos
sellin = pd.read_csv("../datasets/sell-in.txt.gz", sep="\t")
productos = pd.read_csv("../datasets/tb_productos.txt", sep="\t")
productos = productos.drop_duplicates(subset=["product_id"],keep="first")  # Eliminar duplicados por si acaso
stocks = pd.read_csv("../datasets/tb_stocks.txt", sep="\t")

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
del sellin, productos, stocks

# Agrupar ventas por periodo, cliente y producto
dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "cat1":"first",
                                                                "cat2":"first",
                                                                "cat3":"first",
                                                                "brand":"first",
                                                                "sku_size":"first",
                                                                "stock_final":"first",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
# 1. Calcular primeros períodos

periodo_producto =  df.groupby(['periodo', 'product_id'])["tn"].sum().rename('periodo_producto')
nacimiento_producto = df.groupby('product_id')['periodo'].min().rename('nacimiento_producto')
#muerte_cliente = df.groupby('customer_id')['periodo'].max().rename('muerte_cliente')

# 2. Crear todas las combinaciones posibles
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()
periodos = df['periodo'].unique()

# 3. Crear matriz de combinaciones válidas usando broadcasting
idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
# 4 filtrar combinaciones periodo_producto
completo = completo.merge(periodo_producto, on=['periodo', 'product_id'], how='left')
completo = completo[completo['periodo_producto'].notna()]
# 5 Filtrar combinaciones nacimiento_producto
completo = completo.merge(nacimiento_producto, on='product_id', how='left')
completo = completo[completo['periodo'] >= completo['nacimiento_producto']]
completo = completo[completo['nacimiento_producto'] < 201910]

# 6. Filtrar combinaciones muerte_cliente
# completo = completo.merge(muerte_cliente, on='customer_id', how='left')
# completo = completo[completo['periodo'] <= completo['muerte_cliente']]


# 7. Unir con datos reales
df_completo = completo.merge(dt, how='left')
df_completo['tn'] = df_completo['tn'].fillna(0)

# Convertir periodo a datetime con día fijo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear columna periodo_target_dt desplazada 2 meses
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

# DataFrame auxiliar con ventas futuras
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras = ventas_futuras.rename(columns={
    'periodo_dt': 'periodo_target_dt',
    'tn': 'target'
})

# Hacemos el merge usando fechas datetime directamente
dt = pd.merge(
    df_completo,
    ventas_futuras,
    how='left',
    on=['periodo_target_dt', 'customer_id', 'product_id']
)
dt = dt.drop(columns=['periodo_target_dt']) # Ya usamos esta para hacer el merge
del df_completo, ventas_futuras, completo, idx, periodo_producto, nacimiento_producto#, muerte_cliente
gc.collect()

dt.shape

(18581028, 17)

In [2]:
dt[dt['periodo'] <= 201910].shape

(17514786, 17)

In [3]:
#Prueba target
dt[
    (dt['customer_id'] == 10050) &
    (dt['product_id'] == 20001)
][["periodo","tn","target"]].head(50)

,periodo,tn,target
7998033,201701,0.16790,0.00000
7998034,201702,0.16790,0.02239
7998035,201703,0.00000,0.12312
7998036,201704,0.02239,0.17909
7998037,201705,0.12312,0.22386
7998038,201706,0.17909,0.83947
7998039,201707,0.22386,0.24625
7998040,201708,0.83947,0.00000
7998041,201709,0.24625,0.30221
7998042,201710,0.00000,0.12312


# nuevo target delta

In [4]:
dt["target"] = dt["target"]-dt["tn"]

In [5]:
#dt.to_csv("../datasets/dt_target.csv.gz", index=False, sep="\t",compression='gzip')
dt.to_csv("../datasets/dt_target2.csv", index=False, sep=",")

In [ ]:

# crear alguans variables nuevas de prueba
dt["month"] = dt["periodo_dt"].dt.month
dt["year"] = dt["periodo_dt"].dt.year


# Convertir columnas categóricas a tipo 'category' para LightGBM
for col in ['cat1', 'cat2', 'cat3', 'brand', 'plan_precios_cuidados','descripcion']:
    if col in dt.columns:
        dt[col] = dt[col].astype('category')

dt_kgl = dt[dt["periodo"].isin([201912])]
dt = dt.drop(dt[dt["periodo"].isin([201911,201912])].index,axis=0)

In [ ]:
dt.shape

(9349922, 17)

In [ ]:
dt.columns

Index(['product_id', 'customer_id', 'periodo', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'stock_final', 'periodo_dt',
       'periodo_target_dt', 'target'],
      dtype='object')

In [ ]:
# prompt: ajustar un modelo lightgbm de regresion, teniendo como objetivo la columna target

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np



# Columnas a utilizar como features (X) y la columna target (y)
# Se excluyen las columnas de identificación y las que no son numéricas o son el target
feature_columns = [col for col in dt.columns if col not in ['periodo','periodo_dt', 'target']]
X = dt[feature_columns]
y = dt['target']



# Dividir los datos en conjuntos de entrenamiento y prueba (opcional, pero recomendado)
# Para este caso, vamos a entrenar con todos los datos disponibles antes de 201911
# Si quisieras validación, podrías usar un periodo anterior como validación

# Definir el modelo LightGBM
lgb_reg = lgb.LGBMRegressor(random_state=42)

# Entrenar el modelo
lgb_reg.fit(X, y)

print("Modelo LightGBM entrenado con éxito.")

# Ahora puedes usar lgb_reg para hacer predicciones
# Por ejemplo, si tuvieras un dataframe 'X_pred' con los mismos features:
# predictions = lgb_reg.predict(X_pred)



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.798021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2099
[LightGBM] [Info] Number of data points in the train set: 9349922, number of used features: 15
[LightGBM] [Info] Start training from score 0.129384
Modelo LightGBM entrenado con éxito.


In [ ]:
# prompt: calcular accuracy del modelo entrenado

import numpy as np
from sklearn.metrics import accuracy_score

# Para calcular la precisión (accuracy), necesitas tener etiquetas binarias (0 o 1).
# Tu target 'tn' es una variable continua representando ventas.
# Accuracy es adecuada para problemas de clasificación.

# Si quieres evaluar un modelo de regresión como LGBMRegressor,
# deberías usar métricas de regresión como Mean Absolute Error (MAE),
# Mean Squared Error (MSE), R-squared, etc.

# Si deseas adaptar esto a un problema de clasificación (ej: predecir si hay venta o no > 0),
# puedes binarizar el target:
# y_binary = (y > 0).astype(int)
# Si ya has entrenado el modelo con el target continuo, no puedes calcular accuracy directamente.

# Asumiendo que quieres evaluar cómo se desempeña el modelo en los datos de entrenamiento
# (lo cual no es una métrica de rendimiento generalizable, pero ilustra el proceso):

# Hacer predicciones en el conjunto de entrenamiento
y_pred = lgb_reg.predict(X)

# Para un problema de regresión, puedes calcular MAE o MSE
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Si binarizaras el problema a Clasificación (¿Hay venta o no?):
# y_binary = (y > 0).astype(int)
# # Entrenar un clasificador (si es necesario, si el modelo no soporta clasificación binaria)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(random_state=42)
# clf.fit(X, y_binary)
# y_pred_binary = clf.predict(X)
# accuracy = accuracy_score(y_binary, y_pred_binary)
# print(f"Accuracy (binary classification): {accuracy:.4f}")

# Nota: Evaluar en el mismo conjunto de entrenamiento no da una idea realista del rendimiento
# en datos no vistos. Lo ideal sería dividir los datos en entrenamiento y prueba
# ANTES de entrenar y evaluar en el conjunto de prueba.
# O usar validación cruzada.

Mean Absolute Error (MAE): 0.1174
Mean Squared Error (MSE): 1.0259
Root Mean Squared Error (RMSE): 1.0129


In [ ]:


# Columnas a utilizar como features (X) y la columna target (y)
# Se excluyen las columnas de identificación y las que no son numéricas o son el target
X_kgl = dt_kgl[feature_columns]
y_pred = lgb_reg.predict(X_kgl)

In [ ]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": y_pred})
result = result[result["product_id"].isin(productos_ok["product_id"])]
result = result.groupby("product_id").agg({"tn":"sum"}).reset_index()
result

,product_id,tn
0,20001,1445.987290
1,20002,971.359486
2,20003,772.756700
3,20004,516.712323
4,20005,478.195130
...,...,...
775,21263,0.382548
776,21265,0.712950
777,21266,0.712950
778,21267,0.088928


In [ ]:
# prompt: descargar en csv el dataframe result

from google.colab import files
result.to_csv('result.csv', index=False,sep=",")
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>